## Import Library
Import all libraries required to run the functional genetic algorithm

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

## City & Route Class

The City class to make it easier to define cities coordinate and calculate the distance between two cities.

The Route class to determine the route taken and calculate the total distance and fitness value

In [2]:
class City:
    def __init__(self,name, x, y):
        self.x = x
        self.y = y
        self.name = name

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((abs(self.x - city.x) ** 2) + (abs(self.y - city.y)** 2))
        return distance

    def __repr__(self):
        return "("+str(self.name)+")"

In [3]:
class Route:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness = 0.0

    def routeDistance(self):
        if self.distance == 0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None

                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]

                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

    def printCity(self):
        namaKota =[]
        for i in range(0,len(self.route)):
            namaKota.append(self.route[i].name)
        return "("+str(namaKota)+ ")"

## Genetic Algorithm Functions

Create several functions to perform each step in the genetic algorithm, such as:
- Create Route
- Initial Population
- Rank Routes
- Selection
- Check Tabulist
- Crossover
- Mutate
- Update Generation

### Create Route

In [4]:
def createRoute(cityList):
    #Randomize city based on city in dataset to assign into route
    route = random.sample(cityList, len(cityList))
    return route

### Intialization the Population

In [5]:
def initialPopulation(popSize, cityList):
    population = []
    # Create population based on population size
    for i in range(0, popSize):
      route = createRoute(cityList)
      while route in population:
        route = createRoute(cityList)
      population.append(createRoute(cityList))
    return population

### Rank Routes

In [14]:
def rankRoutes(population):
    fitnessResults = []
    rankRoute = {}

    #Determine fitness each individual in population
    for i in range(0, len(population)):
        fitnessResults.append(Route(population[i]).routeFitness())

    #Sorting based on fitness value
    sortedFitness = sorted(fitnessResults, reverse=True)
    for i in range(0, len(sortedFitness)):
        for j in range(0, len(population)):
            if sortedFitness[i] == Route(population[j]).routeFitness():
                rankRoute[i] = population[j]

    return rankRoute

### Selection

In [24]:
def selection(population):
    selection = []
    #Take first 2  the best individuals
    for i in range(0,2):
        selection.append(population[i])
        print("seleksi ke-" + str(i+1) + " " + str(Route(selection[i]).printCity()) + "\n" +
              " Jarak = " + str(Route(selection[i]).routeDistance()) + "\n" + " Fitness = " + str(
            Route(selection[i]).routeFitness()))
    return selection

### Tabulist

In [17]:
def tabulist(selectionResults,tabulist):
    isTabulist1 = False
    isTabulist2 = False
    isTabulist = False

    #Check tabulist
    for j in range(0,len(tabulist)):
        if selectionResults[0] == tabulist[j]:
            isTabulist1 = True
        if selectionResults[1] == tabulist[j]:
            isTabulist2 = True

    if isTabulist1 == False:
        tabulist.append(selectionResults[0])
    if isTabulist2 == False:
        tabulist.append(selectionResults[1])

    if isTabulist1 == True and isTabulist2 == True:
        isTabulist = True

    return isTabulist,tabulist

### Crossover

In [18]:
def crossover(parent1, parent2,tab):
    childP1 = []
    childP2 = []
    child = []

    # randomizes the gene number to be carried out crossover
    # if tabulis is true then the crossover result is the same as parent1 (best individual)
    if tab == True:
        geneA = 0
        geneB = len(parent1)-1
    else:
        geneA = random.randint(1, len(parent1)-1)
        geneB = random.randint(1, len(parent1)-1)
        # Gene A and Gene B do not have the same value
        while geneA == geneB:
            geneB = random.randint(1, len(parent1)-1)

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    print("stargene = "+str(startGene+1)+", endGene = "+str(endGene+1))

    for i in range(startGene, endGene+1):
        childP1.append(parent1[i])

    for item in parent2:
        if item not in childP1:
            childP2.append(item)

    #Crossover Result
    idxChild1 = 0
    idxChild2 = 0
    for i in range(len(parent1)):
      if i >= startGene and i <= endGene:
        child.append(childP1[idxChild1])
        idxChild1 += 1
      else:
        child.append(childP2[idxChild2])
        idxChild2 += 1

    return child

### Mutate

In [19]:
def mutate(individual):
    #randomizes the gene number to be exchanged
    swapped = random.randint(1, len(individual)-1)
    swapWith = random.randint(1, len(individual)-1)
    #swapped and swapWith cannot have the same value
    while swapped == swapWith:
        swapWith = random.randint(1, len(individual)-1)

    city1 = individual[swapped]
    city2 = individual[swapWith]

    print("swapped = "+str(swapped+1) + " , swap with = " + str(swapWith+1))

    #hasil mutasi
    individual[swapped] = city2
    individual[swapWith] = city1
    return individual

### Update Generation

In [20]:
def updateGeneration(population,popsize,mutate):
    generasibaru =[]
    #combining populations with mutation results to create a new generation
    for i in range(0,popsize):
        generasibaru.append(population[i])

    #replace the individual with the smallest fitness (last index) with the mutation result if the mutation result is greater
    if Route(mutate).routeFitness() >= Route(generasibaru[-1]).routeFitness():
      generasibaru[popsize-1] =mutate

    for i in range(0,popsize):
        print("Generasi baru ke-" + str(i+1) + " " + str(Route(generasibaru[i]).printCity()) +
              " Jarak = " + str(Route(generasibaru[i]).routeDistance()) + " Fitness = " + str(
            Route(generasibaru[i]).routeFitness()))

    return generasibaru

### Implement The Genetica Algorithm

In [21]:
def geneticAlgorithm(population, popSize, generations):
    tabulistResult = []
    finalPopulation = []
    pop = []

    # Create Population
    populasi = initialPopulation(popSize, population)

    # Sort each individual in population based on ranking
    rankPopulasi = rankRoutes(populasi)
    for i in range(0, popSize):
        pop.append(rankPopulasi[i])

    # Start genetic algorithm with the first generation
    print("=========================== GENERASI KE " + str(1) + "===============================================================>")
    for i in range(0,len(pop)):
        print("populasi ke-" + str(i+1) + " " + str(Route(pop[i]).printCity()) +
              " Jarak = " + str(Route(pop[i]).routeDistance()) + " Fitness = " + str(
            Route(pop[i]).routeFitness()))
    print("==========================================================================================>")

    bestDistance = 9999999999999
    progress =[]
    for i in range(0, generations):
        nextGeneration = []

        #Selection process
        selectionResults = selection(pop)

        #Check Tabulist
        tab, tabulistResult = tabulist(selectionResults, tabulistResult)

        #Crossover if the tabulist true
        crossoverResult = crossover(selectionResults[0], selectionResults[1], tab)
        if tab == False:
            print("Tabulist False")
            print("==========================================================================================>")
            print("hasil crossover = " + str(crossoverResult))
            print("==========================================================================================>")
        elif tab == True:
            print("Tabulist true")
            print("==========================================================================================>")
            print("seleksi 1 = " + str(crossoverResult))
            print("==========================================================================================>")
        print("TABULIST : ", tabulistResult, "\n")

        #Mutate Process
        children = mutate(crossoverResult)
        print("hasil mutasi = " + str(children) + " Jarak = " + str(Route(children).routeDistance()) + " Fitness = " + str(
            Route(children).routeFitness()))
        print("==========================================================================================>")

        #Update Generation
        nextGeneration = updateGeneration(pop, popSize, children)
        poptemp = nextGeneration

        # Sorting new generations based on fitness
        poptemp = rankRoutes(poptemp)
        pop = []
        for j in range(0, popSize):
            pop.append(poptemp[j])

        # Save the best individual in this generation
        tempbestroute = Route(pop[0]).routeDistance()
        progress.append(Route(pop[0]).routeDistance())
        if (bestDistance > tempbestroute):
            bestIndividu = Route(pop[0]).printCity()
            bestDistance = Route(pop[0]).routeDistance()
            bestFitness = Route(pop[0]).routeFitness()

        # Next Generation process
        finalPopulation = pop
        print("\n====================================== GENERASI KE " + str(
            i + 2) + "======================================>")
        for n in range(0, len(pop)):
            print("populasi ke-" + str(n+1) + " " + str(Route(pop[n]).printCity()) +
                " Jarak = " + str(Route(pop[n]).routeDistance()) + " Fitness = " + str(
                Route(pop[n]).routeFitness()))
        print("=========================================================================================>")


    print("BEST INDIVIDU = " + str(bestIndividu))
    print("BEST DISTANCE = " + str(bestDistance))
    print("BEST FITNESS = " + str(bestFitness))

    # Visualize the genetic algorithm result
    plt.figure(0)
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.title('Genetica Algorithm Result')
    plt.savefig('algen_result.png')

    return finalPopulation

## Main

In [25]:
%%capture cap
DATASET_PATH = "t5.csv"

# Genetic Algorithm Parameters
POPULATION_SIZE = 5
GENERATION = 50

# Read city dataset
city= []
city = pd.read_csv(DATASET_PATH, header=None , sep=' ')

# Define City list
cityList = []
for i in range(0,len(city)):
    cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

# Start Genetic Algorithm process
print("\n====================================== Genetica Algorithm ======================================\n")
print(f"Population Size: {str(POPULATION_SIZE)}")
print(f"Generation: {str(GENERATION)}")
print()
result = geneticAlgorithm(population=cityList, popSize=POPULATION_SIZE, generations=GENERATION)

with open('algen_result.txt', 'w') as f:
    f.write(cap.stdout)